# Fortgeschrittene Konzepte für RNNs
## GRU-Schichten
GRU (Gated Recurrent Unit)-Schichten erfüllen diesselbe Funktion wie eine LSTM-Schicht: sie lassen den Fluss von Informationen aus weit vorangegangen Perioden gewichtet zu.

Der Unterschied ist, dass sie weniger Rechenleistung benötigen, was mit einer kleineren Repäsentationskraft bezahlt wird

## Recurrent Dropout
In der Forschung schien sich zunächst abzubilden, dass Dropout in RNNs eher hinderlich als hilfreich ist. Wird allerdings das gleiche Muster an Dropout auf jede Schicht angewandt, so ändert sich das. So sollte auch das gleiche Muster in allen Matrizen innerhalb einer RNN-Schicht verwendet werden. Dies lässt sich so codieren:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
model = Sequential()
# ...
model.add(GRU(
    32, 
    dropout = 0.2,
    recurrent_dropout = 0.2,
    input_shape = (None, data.shape[-1])
))
# ...

## Mehrere RNN-Schichten hintereinander
Um das zu tun, müssen alle Schichten, die nicht ganz oben liegen `return_sequences = True` aktiviert haben. Es ist ein guter Weg, um ein Netz zu erweitern, um es an die Grenzen des Overfitting zu bringen, und so mehr Repräsentationskraft zu finden.